In [2]:
from sklearn import metrics
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score

import pandas as pd
import numpy as np

The structure data is in a variety of formats; ultimately I'd say it's such that it is categorical since the output is whether the client made a purchase or not.

In [93]:
bank = pd.read_csv("bank.csv")
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [94]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4521 entries, 0 to 4520
Data columns (total 17 columns):
age          4521 non-null int64
job          4521 non-null object
marital      4521 non-null object
education    4521 non-null object
default      4521 non-null object
balance      4521 non-null int64
housing      4521 non-null object
loan         4521 non-null object
contact      4521 non-null object
day          4521 non-null int64
month        4521 non-null object
duration     4521 non-null int64
campaign     4521 non-null int64
pdays        4521 non-null int64
previous     4521 non-null int64
poutcome     4521 non-null object
y            4521 non-null object
dtypes: int64(7), object(10)

In [95]:
job_dummies = pd.get_dummies(bank.job, prefix='job').ix[:,1:]
marital_dummies = pd.get_dummies(bank.marital, prefix='married').ix[:,1:]
default_dummies = pd.get_dummies(bank.default, prefix='default').ix[:,1:]
poutcome_dummies = pd.get_dummies(bank.poutcome, prefix='poutcome').ix[:,1:]
month_dummies = pd.get_dummies(bank.month, prefix='month').ix[:,1:]
contact_dummies = pd.get_dummies(bank.contact, prefix='contact').ix[:,0:]
y_dummies = pd.get_dummies(bank.y, prefix='y').ix[:,0:]


bank = pd.concat([bank, job_dummies], axis=1)
bank = pd.concat([bank, marital_dummies], axis=1)
bank = pd.concat([bank, default_dummies], axis=1)
bank = pd.concat([bank, poutcome_dummies], axis=1)
bank = pd.concat([bank, contact_dummies], axis=1)
bank = pd.concat([bank, month_dummies], axis=1)
bank = pd.concat([bank, y_dummies], axis=1)

bank.describe()

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,y_no,y_yes
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,...,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579,0.209246,0.037160,0.024773,...,0.032736,0.156160,0.117452,0.010838,0.309224,0.086043,0.017695,0.011502,0.884760,0.115240
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562,0.406815,0.189174,0.155451,...,0.177965,0.363047,0.321994,0.103553,0.462225,0.280458,0.131856,0.106640,0.319347,0.319347
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [96]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4521 entries, 0 to 4520
Data columns (total 50 columns):
age                  4521 non-null int64
job                  4521 non-null object
marital              4521 non-null object
education            4521 non-null object
default              4521 non-null object
balance              4521 non-null int64
housing              4521 non-null object
loan                 4521 non-null object
contact              4521 non-null object
day                  4521 non-null int64
month                4521 non-null object
duration             4521 non-null int64
campaign             4521 non-null int64
pdays                4521 non-null int64
previous             4521 non-null int64
poutcome             4521 non-null object
y                    4521 non-null object
job_blue-collar      4521 non-null float64
job_entrepreneur     4521 non-null float64
job_housemaid        4521 non-null float64
job_management       4521 non-null float64
job_retired   

In [97]:
feature_cols = ['age',"contact_telephone","contact_cellular","job_blue-collar","job_entrepreneur","job_housemaid","job_management",
                "job_retired","job_self-employed","job_services","job_student","job_technician",
                "job_unemployed","job_unknown","married_married","married_single","default_yes",
                "poutcome_other","poutcome_success","poutcome_unknown","month_aug","month_dec",
                "month_feb","month_jan","month_jul","month_jun","month_mar","month_may","month_nov",
                "month_oct","month_sep"]
X = bank[feature_cols]
Y = bank.y_yes.values

In [98]:
model = DecisionTreeClassifier(max_depth = 3, random_state=0)
scores = cross_val_score(model,X,Y,cv=5,scoring = 'accuracy')
np.mean(scores)

0.89073338874492747

In [99]:
model.fit(X,Y)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=3, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=0, splitter='best')

In [100]:
#model.feature_importances_
pd.DataFrame({'features':feature_cols, 'importance':model.feature_importances_}).sort('importance', ascending = False)

,features,importance
18,poutcome_success,0.667583
29,month_oct,0.141826
0,age,0.124527
6,job_management,0.025800
3,job_blue-collar,0.022387
27,month_may,0.017878
17,poutcome_other,0.000000
28,month_nov,0.000000
26,month_mar,0.000000
25,month_jun,0.000000


In [ ]:
Regarding the accuracy of the model, the cross validation score is 0.89, which is on the higher side, but this could be in part due to the fact that some of the variables that may be irrelevant were dropped. 

In [3]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [4]:
bank2 = pd.read_csv("bank-additional.csv",sep=';')
bank2.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


In [5]:
cat_columns = ["job","marital","education","default","housing","loan","contact","month","day_of_week","pdays","poutcome"]
cat_dict = bank2[cat_columns].to_dict(outtype="records")
cat_dict[:1]

[{'contact': 'telephone',
  'day_of_week': 'mon',
  'default': 'no',
  'education': 'basic.4y',
  'housing': 'no',
  'job': 'housemaid',
  'loan': 'no',
  'marital': 'married',
  'month': 'may',
  'pdays': 999,
  'poutcome': 'nonexistent'}]

In [6]:
from sklearn import feature_extraction
from __future__ import print_function
vec = feature_extraction.DictVectorizer()
cat_vector = vec.fit_transform(cat_dict).toarray()
cat_vector[:1]

array([[   0.,    1.,    0.,    1.,    0.,    0.,    0.,    1.,    0.,
           0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           1.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,
           0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    1.,    0.,    0.,    0.,  999.,    0.,    1.,    0.]])

In [7]:
bank2_vector = pd.DataFrame(cat_vector)
vector_columns = vec.get_feature_names()
#vector_columns

In [8]:
bank2_vector.columns = vector_columns
bank2_vector.index = bank2.index
bank2 = bank2.drop(cat_columns, axis=1)
bank2 = bank2.join(bank2_vector)
bank2['y'] = bank2.y.map({'no':0, 'yes':1})

bank2.head()

,age,duration,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,...,month=jun,month=mar,month=may,month=nov,month=oct,month=sep,pdays,poutcome=failure,poutcome=nonexistent,poutcome=success
0,56,261,1,0,1.1,93.994,-36.4,4.857,5191,0,...,0,0,1,0,0,0,999,0,1,0
1,57,149,1,0,1.1,93.994,-36.4,4.857,5191,0,...,0,0,1,0,0,0,999,0,1,0
2,37,226,1,0,1.1,93.994,-36.4,4.857,5191,0,...,0,0,1,0,0,0,999,0,1,0
3,40,151,1,0,1.1,93.994,-36.4,4.857,5191,0,...,0,0,1,0,0,0,999,0,1,0
4,56,307,1,0,1.1,93.994,-36.4,4.857,5191,0,...,0,0,1,0,0,0,999,0,1,0


In [9]:
bank2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 64 columns):
age                              41188 non-null int64
duration                         41188 non-null int64
campaign                         41188 non-null int64
previous                         41188 non-null int64
emp.var.rate                     41188 non-null float64
cons.price.idx                   41188 non-null float64
cons.conf.idx                    41188 non-null float64
euribor3m                        41188 non-null float64
nr.employed                      41188 non-null float64
y                                41188 non-null int64
contact=cellular                 41188 non-null float64
contact=telephone                41188 non-null float64
day_of_week=fri                  41188 non-null float64
day_of_week=mon                  41188 non-null float64
day_of_week=thu                  41188 non-null float64
day_of_week=tue                  41188 non-null float64
day_o

In [14]:
f_cols = ['age', 'duration', 'campaign', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 
     'contact=cellular', 'contact=telephone', 'day_of_week=fri', 'day_of_week=mon', 'day_of_week=thu', 'day_of_week=tue', 
     'day_of_week=wed', 'default=no', 'default=unknown', 'default=yes', 'education=basic.4y', 'education=basic.6y', 
     'education=basic.9y', 'education=high.school', 'education=illiterate', 'education=professional.course', 
     'education=university.degree', 'education=unknown', 'housing=no', 'housing=unknown', 'housing=yes', 'job=admin.', 
     'job=blue-collar', 'job=entrepreneur', 'job=housemaid', 'job=management', 'job=retired', 'job=self-employed', 
     'job=services', 'job=student', 'job=technician', 'job=unemployed', 'job=unknown', 'loan=no', 'loan=unknown', 
     'loan=yes', 'marital=divorced', 'marital=married', 'marital=single', 'marital=unknown', 'month=apr', 'month=aug', 
     'month=dec', 'month=jul', 'month=jun', 'month=mar', 'month=may', 'month=nov', 'month=oct', 'month=sep', 'pdays',
     'poutcome=failure', 'poutcome=nonexistent', 'poutcome=success']
X= bank2[f_cols]
Y = bank2.y.values



In [15]:
model = DecisionTreeClassifier(max_depth = 3, random_state=0)
scores = cross_val_score(model,X,Y,cv=5,scoring = 'accuracy')
model.fit(X,Y)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=3, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=0, splitter='best')

In [16]:
pd.DataFrame({'features':f_cols, 'importance':model.feature_importances_}).sort('importance', ascending = False)

,features,importance
1,duration,0.505931
8,nr.employed,0.409071
6,cons.conf.idx,0.037775
59,pdays,0.034946
62,poutcome=success,0.012276
40,job=unemployed,0.000000
45,marital=divorced,0.000000
44,loan=yes,0.000000
43,loan=unknown,0.000000
42,loan=no,0.000000


In [17]:
np.mean(scores)

0.57377392194924093

In [ ]:
Repeating the analysis, this time using all the variables, and the OneHotEncoder, the performance of the model seems to have gotten worse possibly due to the addition of many other variables that may not be as deterministic.